# Demo of core features

First, some imports and some plot settings.

In [14]:
import glob
import imageio.v2 as iio
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import yaml

from collections import defaultdict
from importlib import reload
from itertools import product
from numpy.random import Generator
from pathlib import Path
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from time import time
from tqdm.notebook import tqdm

from genexpy import lower_bounds as gu
from genexpy import kernels as ku
from genexpy import probability_distributions as prob
from genexpy import rankings_utils as ru
from genexpy import mmd as mmd

mpl.use("TkAgg")
sns.set_style("ticks")
sns.set_context("notebook")
palette = "flare_r"
sns.set_palette("flare_r")

Next, we load the hyperparameters from the config.

In [15]:
with open("config.yaml", 'r') as file:
    config = yaml.safe_load(file)

FIGURES_DIR = Path("figures_global")

FORMAT = config['format']['output']

SEED = config['parameters']['seed']
RNG = np.random.default_rng(SEED)
EPS = config['parameters']['epsilon']
ALPHA = config['parameters']['alpha']
LR_CONFIDENCE = config['parameters']['lr_confidence']
CI_LOWER = (1 - LR_CONFIDENCE) / 2
CI_UPPER = LR_CONFIDENCE + CI_LOWER

DATASET = Path(config['data']['dataset_path'])
EXPERIMENTAL_FACTORS = config['data']['experimental_factors']
TARGET = config['data']['target']
ALTERNATIVES = config['data']['alternatives']

SAMPLE_SIZE = config['sampling']['sample_size']
DISJOINT = config['sampling']['disjoint']
REPLACE = config['sampling']['replace']

Then, we load the dataset into a pandas dataframe and we check that only one experimental factor is set to `None`, indicating that it is allowed to vary. 

In [16]:
if DATASET.suffix == '.parquet':
    df = pd.read_parquet(DATASET).drop(columns=["time"])
elif DATASET.suffix == '.csv':
    df = pd.read_csv(DATASET).drop(columns=["time"])
else:
    raise Exception("Please use a Parquet or CSV file as the format of your data")

# Check whether exactly one of the experimental factors is None 
assert sum(value is None for value in EXPERIMENTAL_FACTORS.values()) == 1, "Exactly one experimental factor must be None"

columns_to_check = set(EXPERIMENTAL_FACTORS.keys()).union({TARGET, ALTERNATIVES})
# Check whether the factors listed in the config actually exist in the df
if not_in_df := columns_to_check - set(df.columns):
    raise ValueError(f"The following columns are missing from the dataframe: {not_in_df}")

# Check whether the factors listed in the config are exhaustive
if not_in_config:= set(df.columns) - columns_to_check:
    raise ValueError(f"The following columns in the dataframe are not required: {not_in_config}")

We then build a query from the experimental factors and convert the `df` to a rank matrix. Note that here, we assume that we are using the _all key to designate that we want to use all possible values for that column.

In [17]:
all_factor_key = next((k for k, v in EXPERIMENTAL_FACTORS.items() if v == "_all"), None)
dataframes_list = []

if all_factor_key:
    unique_values = df[all_factor_key].unique()
    for value in unique_values:
        current_factors = EXPERIMENTAL_FACTORS.copy()
        current_factors[all_factor_key] = value
        query_string = " and ".join(f"{key} == '{val}'" if isinstance(val, str) else f"{key} == {val}"
                                    for key, val in current_factors.items() if val is not None)
        subset_df = df.query(query_string).reset_index(drop=True)
        dataframes_list.append(subset_df)
else:
    query_string = " and ".join(f"{key} == '{val}'" if isinstance(val, str) else f"{key} == {val}"
                                for key, val in EXPERIMENTAL_FACTORS.items() if val is not None)
    single_df = df.query(query_string).reset_index(drop=True)
    dataframes_list.append(single_df)

Next, we load the kernels that we defined in the config. We will later iterate over them, as to perform our analysis over all 

In [18]:
def init_kernels(rank_matrix):
    kernels = {}
    for kernel_config in config['kernels']:
        kernel_func = getattr(ku, kernel_config['kernel'], None)
        
        if kernel_func:
            for param_key, param_values in kernel_config['params'].items():
                if isinstance(param_values, list):
                    for value in param_values:
                        params = {param_key: value}
                        if param_key == 'idx':
                            params[param_key] = rank_matrix.index.get_loc(value)
                        
                        kernel_name = f"{kernel_config['kernel']}_{param_key}_{value}"
                        kernels[kernel_name] = (kernel_func, params)
                else:
                    params = {param_key: param_values}
                    if param_key == 'idx':
                        params[param_key] = rank_matrix.index.get_loc(param_values)

                    kernel_name = f"{kernel_config['kernel']}_{param_key}_{param_values}"
                    kernels[kernel_name] = (kernel_func, params)
        else:
            print(f"Kernel function '{kernel_config['kernel']}' not found in module 'kernels'.")
    return kernels

We create some directories for the different experiments we run.

In [19]:
def create_experiment_directory(name, factors):
    exp0_dir_name_parts = [f"name={ALTERNATIVES}"] + [f"{key}={value}" for key, value in factors.items() if value is not None]
    
    exp0_dir = FIGURES_DIR / "_".join(exp0_dir_name_parts)
    
    exp1_dir = exp0_dir / f"{name}"
    
    exp2_dir = exp1_dir / f"nstar_N_ALPHA={ALPHA}_eps={EPS}_ci={LR_CONFIDENCE}_disjoint={DISJOINT}_replace={REPLACE}"
    
    exp2_dir.mkdir(parents=True, exist_ok=True)

    return exp2_dir

We will now define a few methods that will be run in a loop until our `ec_pool` is empty.

We start by sampling from our `ec_pool` and converting our samples to the corresponding rankings.

In [20]:
def sample_ecs(ec_pool, sample_size):
    # Safety check such that we do not take more samples than we can take
    sample_size = min(sample_size, len(ec_pool))

    # Sample experimental conditions
    return RNG.choice(ec_pool, sample_size, replace=False)

def compute_rankings(ecs, rank_matrix):
    rm_ = rank_matrix.loc[:, ecs]
    na, nv = rm_.shape
    
    # Generate rankings from the data
    rankings = ru.SampleAM.from_rank_function_dataframe(rm_)
    
    return rankings, nv

We then get compute the variance, variance lower bound and MMDs of these rankings.

In [21]:
def compute_variance_and_lower_bound(rankings, n, kbar, kernel, kernelargs):
    
    variance = ku.var(rankings, use_rv=True, kernel=kernel, **kernelargs)
    
    var_lower_bound = gu.sample_mean_embedding_lowerbound(EPS, n, kbar=1, v=variance)
    
    return variance, var_lower_bound

def calculate_mmds(rankings, nv, kernel, kernelargs):
    mmds = {
        n: mmd.subsample_mmd_distribution(
            rankings, subsample_size=n, rep=100, use_rv=True, use_key=False,
            seed=SEED, disjoint=DISJOINT, replace=REPLACE, kernel=kernel, **kernelargs
        )
        for n in range(2, nv // 2 + 1)
    }
    return mmds

Then, we create dataframes consisting of Generalizability scores and Quantiles.

In [22]:
def create_generalizability_dataframe(mmds, logepss):
    ys = {n: [mmd.generalizability(mmde, np.exp(logeps)) for logeps in logepss] for n, mmde in mmds.items()}
    dfy = pd.DataFrame(ys, index=logepss).reset_index().melt(id_vars='index', var_name='n', value_name='generalizability')
    dfy.rename(columns={'index': 'log(eps)'}, inplace=True)
    dfy['n'] = dfy['n'].astype(int)
    return dfy

def create_quantiles_dataframe(mmds):
    qs = {n: np.log(np.quantile(mmde, ALPHA)) for n, mmde in mmds.items()}
    dfq = pd.DataFrame(list(qs.items()), columns=['n', 'log(eps)'])
    dfq['log(n)'] = np.log(dfq['n'])
    return qs, dfq


We now use the quantiles dataframe to calculate `nstar`. To this end, we fit linear regression models on subsets.

In [23]:
def perform_linear_regression_with_cv(dfq):
    # Extracting features and target from DataFrame
    X = dfq[['log(eps)']].values
    y = dfq[['log(n)']].values

    cv = KFold(n_splits=len(y))

    residuals, linear_predictors = [], []

    for train_index, test_index in cv.split(X):
        lr = LinearRegression().fit(X[train_index], y[train_index])

        predicted = lr.predict(X[test_index])
        residuals.extend(y[test_index] - predicted)

        linear_predictors.append(lr)

    return linear_predictors, residuals

def predict_nstar(logepss, linear_predictors, dfq):
    X = dfq[['log(eps)']].values
    y = dfq[['log(n)']].values

    ns_pred_cv = [np.exp(lr.predict(logepss.reshape(-1, 1)).reshape(-1)) for lr in linear_predictors]
    
    ns_pred = np.exp(LinearRegression().fit(X, y).predict(logepss.reshape(-1, 1)).reshape(-1))
    
    nstar_cv = [pred[np.argmax(logepss > np.log(EPS))] for pred in ns_pred_cv if not np.all(pred == 0)]
    
    nstar = ns_pred[np.argmax(logepss > np.log(EPS))]
    
    nstar_lower, nstar_upper = np.quantile(nstar_cv, [CI_LOWER, CI_UPPER])

    return ns_pred, ns_pred_cv, nstar, nstar_lower, nstar_upper

Lastly, we plot the our results.

In [24]:
def plot_generalizability_and_quantiles(dfy, dfq, logepss, ns_pred, ns_pred_cv, mmds, qs, nstar, nstar_upper, nstar_lower, kernel_dir):
    # Create figure and axes
    fig, axes = plt.subplots(2, 1, sharex="all", figsize=(10, 8))

    # Generalizability plot
    ax = axes[0]
    sns.lineplot(data=dfy, x="log(eps)", y="generalizability", hue="n", ax=ax, palette=palette)
    ax.hlines(ALPHA, ls="--", xmin=np.min(logepss), xmax=np.max(logepss), color="black")
    for n in mmds.keys():
        ax.vlines(qs[n], ymin=0, ymax=ALPHA, ls=":")
    sns.despine(ax=ax)

    # Quantiles plot
    ax = axes[1]
    ymax = max(ns_pred)
    sns.lineplot(data=dfq, x="log(eps)", y="n", ax=ax, ls="", marker="o", hue="n", legend=False)
    for n in mmds.keys():
        ax.vlines(qs[n], ymin=n, ymax=ymax, ls=":")
    ax.vlines(np.log(EPS), ymin=0.1, ymax=ymax, color="black", ls="--")
    sns.lineplot(x=logepss, y=ns_pred, color="green", ls="-.", ax=ax)
    for it, ns_tmp in enumerate(ns_pred_cv):
        if np.max(ns_tmp) > 1000:
            continue
        sns.lineplot(x=logepss, y=ns_tmp, color="green", ls="-.", alpha=0.5, ax=ax)

    # N* Lines
    ax.hlines(nstar, xmin=np.min(logepss), xmax=np.log(EPS), ls="-", color="red")
    ax.hlines(nstar_upper, xmin=np.min(logepss), xmax=np.log(EPS), ls="-", color="red", alpha=0.3)
    ax.hlines(nstar_lower, xmin=np.min(logepss), xmax=np.log(EPS), ls="-", color="red", alpha=0.3)
    ax.set_yscale("log")
    sns.despine(ax=ax)

    # Finalize and save
    fig.suptitle(f"Generalizability for N = {len(ecs):02d}\n"
                 f"n*(alpha={ALPHA}, eps={EPS}) = {np.ceil(nstar)}\n"
                 f"{LR_CONFIDENCE} confidence interval: [{np.ceil(nstar_lower)}, {np.ceil(nstar_upper)}]")
    plt.tight_layout()
    if FORMAT == "pdf" or FORMAT == "all":
        plt.savefig(kernel_dir / f"N={len(ecs):02d}.pdf")
    if FORMAT == "png" or FORMAT == "all":
        plt.savefig(kernel_dir / f"N={len(ecs):02d}.png")
    plt.close("all")

In [25]:
plt.ioff()

for idf in tqdm(dataframes_list, desc='Configurations', leave=True):
    factors_dict = {factor: idf[factor].unique()[0] for factor in EXPERIMENTAL_FACTORS}
    # -- convert df to rank matrix
    rank_matrix = ru.get_rankings_from_df(idf, factors=list(EXPERIMENTAL_FACTORS.keys()), 
                                            alternatives=ALTERNATIVES,
                                            target=TARGET,
                                            lower_is_better=False, impute_missing=True)
    rank_matrix = rank_matrix.fillna(rank_matrix.max())

    # -- get all kernels
    kernels = init_kernels(rank_matrix)

    # -- set up the ec pool
    ec_variable = next((key for key, value in EXPERIMENTAL_FACTORS.items() if value is None), None)
    ec_pool = idf[ec_variable].unique()
    ecs = np.array([])
    total_iterations = (len(ec_pool) + 9) // SAMPLE_SIZE

    for kernelname, (kernel, kernelargs) in tqdm(kernels.items(), desc='Kernels', leave=False):
        kernel_dir = create_experiment_directory(kernelname, factors_dict)
        out = []
        for i in tqdm(range(total_iterations), desc=f'Using {kernelname}', leave=False):
            # -- Sample new rankings from ec pool
            ecs = sample_ecs(ec_pool, (i+1)*SAMPLE_SIZE)
            rankings, nv = compute_rankings(ecs, rank_matrix)

            # -- Compute the lower bound
            variance, var_lower_bound = compute_variance_and_lower_bound(rankings, n=len(ecs), kbar=1, kernel=kernel, kernelargs=kernelargs)

            # -- Compute mmds
            mmds = calculate_mmds(rankings, nv, kernel=kernel, kernelargs=kernelargs)

            # -- Compute generalizability and quantiles

            # Prepare log(eps) scale
            logepss = np.linspace(np.log(EPS) - 0.1, np.log(max(np.quantile(mmde, ALPHA) for mmde in mmds.values())) + 0.1, 1000)

            # Dataframe for generalizability
            dfy = create_generalizability_dataframe(mmds, logepss)

            # Dataframe for quantiles
            qs, dfq = create_quantiles_dataframe(mmds)

            # Linear Regression with Cross-Validation
            linear_predictors, residuals = perform_linear_regression_with_cv(dfq)
            
            # Predictions
            ns_pred, ns_pred_cv, nstar, nstar_lower, nstar_upper = predict_nstar(logepss, linear_predictors, dfq)

            # -- Plotting
            plot_generalizability_and_quantiles(dfy, dfq, logepss, ns_pred, ns_pred_cv, mmds, qs, nstar, nstar_upper, nstar_lower, kernel_dir)

            out.append({
                "kernel": kernelname,
                "alpha": ALPHA,
                "eps": EPS,
                "disjoint": DISJOINT,
                "replace": REPLACE,
                "N": len(ecs),
                "nstar": nstar,
                "nstar_lower": nstar_lower,
                "nstar_upper": nstar_upper,
                "variance": variance,
                "var_lower_bound": var_lower_bound,
            })
        if FORMAT == "gif" or FORMAT == "all":
            images = [iio.imread(image) for image in glob.glob(str(kernel_dir / "*.png"))]
            iio.mimwrite(kernel_dir / f"nstar.gif", images, duration=750,
                            loop=0)
        # -- Store nstar predictions
        out = pd.DataFrame(out)
        out.to_parquet(kernel_dir / "nstar.parquet")
plt.ion()



Configurations:   0%|          | 0/3 [00:00<?, ?it/s]

Kernels:   0%|          | 0/4 [00:00<?, ?it/s]

Using mallows_kernel_nu_auto:   0%|          | 0/3 [00:00<?, ?it/s]

invalid command name "133075143531520process_stream_events"
    while executing
"133075143531520process_stream_events"
    ("after" script)
can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


Using jaccard_kernel_k_1:   0%|          | 0/3 [00:00<?, ?it/s]

...

Using borda_kernel_idx_OHE:   0%|          | 0/3 [00:00<?, ?it/s]

Kernels:   0%|          | 0/4 [00:00<?, ?it/s]

Using mallows_kernel_nu_auto:   0%|          | 0/5 [00:00<?, ?it/s]

Using jaccard_kernel_k_1:   0%|          | 0/5 [00:00<?, ?it/s]

KeyboardInterrupt: 